<center><a href="https://rapids.ai/"><img src="img/rapids.png" alt="rapids_logo" style="width: 1500px;"/></a></center>

# <font color='#8735fb'> Inference

In the previous exercises, we processed our data and trained XGBoost and Random Forest machine learning models.
    
In this Notebook, we will take those models and perform inference. Inference, also referred to commonly as model deployment, edge deployment, or simply "production", is the phase of the machine learning lifecycle where we finally put our model to use. We will be writing some code that loads our model into some sort of application, takes data as input, and returns a prediction or series of predictions as output.
    
For this model, we will simply be using the `test` dataset we split off earlier and running the `.predict()` function inline. In a full machine learning application, you would either embed these `.predict()` calls into some sort of API or Flask application, or better yet, load your model into a robust inference server, such as [Triton](https://developer.nvidia.com/nvidia-triton-inference-server).
    
    
We will be loading each model in two ways (giving us four loaded models in total):
* Using the GPU-accelerated Forest Inference Library (FIL)
* Using the built-in CPU model format
    
After loading these models, we will make predictions, this time benchmarking how long it takes. Note the levels of accuracy we are able to achieve, but also note how long it takes. When running your code, you might not care much about 100 ms versus 200 ms. But if we are running a web application serving thousands of requests per second, the throughput matters, and slower model prediction means more latency or more inference servers. Possibly more crucially, if your model is being used in a real-time embedded system, a high level of latency is unacceptable and can become a matter of safety, life, or death even when dealing with medical equipment, automobiles, or robotics. For these use cases, scaling up or out is not possible, and the main ways to achieve lower latency is with a simpler model (thus giving up on accuracy) or a faster model (using GPU acceleration!).
    
Note, in this context `latency` means the amount of time that elapses between when the input comes in and the prediction is made. We want this to be as small as possible. `throughput` is the number of predictions that can be made in a certain amount of time, say, 1 second. We want this to be as large as possible.

In [ ]:
import cuml
import cudf

from cuml.model_selection import train_test_split

import xgboost
import pickle

In [ ]:
# Define some global directory/file names that change based on the environment

# Set data directory as configured in the workflow
data_dir = '/home/ec2-user/SageMaker/data/'

# Set the name for the previously processed files
processed_data_file = data_dir + 'airlines_processed_data.csv'
processed_data_file_full = data_dir + 'airlines_processed_data_full.csv'

# Specify the path to the model file
xgb_model_file = data_dir + 'xgb_trained_model.model'
rf_model_file = data_dir + 'rf_trained_model.pkl'

In [ ]:
# These are all the columns in the training dataset
train_columns = ['enc_Flight_Number_Reporting_Airline', 'dest_long', 'enc_ArrTime', 'enc_Origin', 'Quarter',\
                 'dest_elevation', 'enc_Dest', 'origin_elevation', 'enc_OriginCityName', 'AirTime', 'DestCityMarketID',\
                 'origin_lat', 'Month', 'enc_DepTime', 'DepDelay', 'enc_Airline', 'enc_CRSArrTime', 'DayOfWeek',\
                 'enc_DestCityName', 'enc_CRSDepTime', 'enc_Reporting_Airline', 'origin_long', 'Distance', 'DepDel15',\
                 'dest_lat', 'OriginCityMarketID']

# This is the column the model will predict
predict_column =  ['ArrDel15']

processed_data = cudf.read_csv(processed_data_file_full)

# Split the preprocessed data into training data and testing data
train_test_split = cuml.preprocessing.model_selection.train_test_split

X_train, X_test, \
y_train, y_test = train_test_split(X = processed_data[train_columns],  
                                   y = processed_data[predict_column], 
                                   train_size = 0.80, random_state = 42)
print(f'Full dataset has {processed_data.shape[0]} rows with {processed_data.shape[1]} columns')
print(f'Training dataset has {X_train.shape[0]} rows with {X_train.shape[1]} columns')
print(f'Test dataset has {X_test.shape[0]} rows with {X_test.shape[1]} columns')

## <font color='#8735fb'> Forest Inference Library (FIL)
The Forest Inference Library is used to load saved forest models of XGBoost or LightGBM, and perform inference on them. It can be used to perform both classification and regression. In this Notebook, we will begin by fitting a model with XGBoost and saving it. We will then load the saved model into FIL and use it to infer on new data.

For additional information on the Forest Inference Library, please refer to the [documentation](https://docs.rapids.ai/api/cuml/stable/api.html#forest-inferencing), and for additional Notebooks, see the [GitHub](https://github.com/rapidsai/cuml/blob/branch-0.20/notebooks/forest_inference_demo.ipynb).
    
To help build some intuition around how this works, think about the format of a Random Forest or a Gradient Boosted Tree. These are ensemble learners, meaning they are models made up of many smaller models. In this case, we trained an XGBoost model that is made up of 300 decision trees. Each of those decision trees is 100 percent independent, and for the XGBoost model to work, it must calculate the output of each tree and then combine it at the end. Does this problem sound familiar? Well, it should! This is exactly the sort of parallel problem that GPUs excel at. We can load the data into GPU memory once, kick off 300 threads all powered by CUDA, and run them simultaneously!
    
You can imagine that with multiple cores or multiple CPUs, this doesn't matter much for smaller models. In fact, try training an XGBoost model with only 10 estimators and it may actually perform worse on FIL. But as you will see in a few cells, for larger (and thus more accurate) models, the Forest Inference Library can give us an order-of-magnitude performance increase.

In [ ]:
from cuml import ForestInference

### <font color='#8735fb'> Load the Pretrained Model
    
The load function of the ForestInference class accepts the following parameters:

       filename : str
           Path to saved model file in a treelite-compatible format
           (See https://treelite.readthedocs.io/en/latest/treelite-api.html
        output_class : bool
           If true, return a 1 or 0 depending on whether the raw prediction
           exceeds the threshold. If False, just return the raw prediction.
        threshold : float
           Cutoff value above which a prediction is set to 1.0
           Only used if the model is classification and output_class is True
        algo : string name of the algo from (from algo_t enum)
             'NAIVE' - simple inference using shared memory
             'TREE_REORG' - similar to naive but trees rearranged to be more
                              coalescing-friendly
             'BATCH_TREE_REORG' - similar to TREE_REORG but predicting
                                    multiple rows per thread block
        model_type : str
            Format of saved treelite model to load.
            Can be 'xgboost', 'lightgbm'
    
Use FIL to load the saved XGBoost model.

In [ ]:
xgb_fil_model = ForestInference.load(
    filename=xgb_model_file,
    algo='BATCH_TREE_REORG',
    output_class=True,
    threshold=0.50,
    model_type='xgboost')

### <font color='#8735fb'> Predict Using FIL
    
After loading  the model, we can simply call `.predict()`, just as we would with any other model format. If we were writing a Flask API or something similar, this is where the bulk of the work happens.

In [ ]:
%%time
# perform prediction on the model loaded from path
fil_preds = xgb_fil_model.predict(X_test)
score = cuml.accuracy_score(y_test, fil_preds)
print( f'XGBoost accuracy score with FIL (GPU) : {score}') 

### <font color='#8735fb'> Non-FIL Inference

FIL is not required to load a model for inference, but it will take advantage of the GPU and run faster in most cases. In the following cell, we load the XGBoost and Random Forest models we previously trained without the use of FIL and without taking advantage of the GPU.
    
Note that for XGBoost, we use the built-in load_model function ans for the Random Forest, we serialized the model to a file and then loaded it back. Because we used the `pickle` package and not another FIL-supported format, we cannot load this directly from the file into FIL. Refer to the [GitHub example Notebooks](https://github.com/rapidsai/cuml/blob/branch-21.08/notebooks/forest_inference_demo.ipynb) or the [cuDF documentation](https://docs.rapids.ai/api/cudf/stable) where we can see how to convert this serialized format to cuDF and then load it into FIL.

In [ ]:
# Load XGBoost model without FIL
xgb_model = xgboost.XGBClassifier()
xgb_model.load_model(xgb_model_file)

In [ ]:
%%time
# perform prediction on the model loaded from path
xgb_preds = xgb_model.predict(X_test)
score = cuml.accuracy_score(y_test, fil_preds)
print( f'XGBoost accuracy score without FIL: {score}') 

If you review the results, you should be seeing something along the lines of 1 second for inference without FIL, compared to 400 ms for inference with FIL. Of course, this is done with the same model, so we should be seeing exactly the same performance. As a side note, this lab is currently running on an NVIDIA T4 GPU, this same benchmark on a V100 GPU will run in 100ms, and on an A100 GPU will run even faster.

In [ ]:
%%time
# Load the previously trained model to make prediction
rf_trained_model = pickle.load(open(rf_model_file, "rb"))
# Use the trained model to predict all the test data
predictions = rf_trained_model.predict(processed_data[train_columns].astype('float32'))
print(f'Predicting {sum(predictions.values_host)} late flights out of {len(predictions)}\n')


### <font color='#8735fb'> Final Benchmark

    
When running inference or doing benchmarks, sometimes the initial period of loading data takes an extra amount of time, or sometimes, you will encounter a problematic batch.
    
To verify the results we saw earlier, we will not repeat the FIL (GPU) and non-FIL (CPU) inference workloads 100 times in a row. This will give us a nicer feel for the average inference performance and, as you'll see, the difference between the two models really adds up as time goes on.

In [ ]:
%%time

for i in range(100):
    fil_preds = xgb_fil_model.predict(X_test)

In [ ]:
%%time

for i in range(100):
    xgb_preds = xgb_model.predict(X_test)

### <font color='#8735fb'> SageMaker Endpoint Deployment

Now that we have our model, know how to use it locally, and understand the performance, we can use SageMaker to [deploy](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-deployment.html) the model to AWS.
    
This gives us an easy API endpoint. With an API endpoint we can integrate our model into the real integration for production use, A/B testing, or general full-stack application development.

In [ ]:
# Initialize the sagemaker session and some other variabls
import sagemaker

# Create a session
execution_role = sagemaker.get_execution_role()
session = sagemaker.Session()

# Get the account/region information
account = !(aws sts get-caller-identity --query Account --output text)
region = !(aws configure get region)

# Specify the image
image_base = 'cloud-ml-sagemaker'
image_tag  = rapids_base_container.split(':')[1]
ecr_fullname = f"{account[0]}.dkr.ecr.{region[0]}.amazonaws.com/{image_base}:{image_tag}"
print(f'Prediction requests can be sent to the following API endpoint: {ecr_fullname}')

# Set endpoint size
instance_type = 'ml.g4dn.xlarge'

In [ ]:
# Create and initialize a predictor
endpoint_model = sagemaker.model.Model(image_uri="ecr_fullname", role=execution_role)
endpoint_model.deploy(initial_instance_count=1, instance_type=instance_type)

predictor = sagemaker.predictor.Predictor(
    endpoint_name=str(endpoint_model.endpoint_name), 
    sagemaker_session=session)

In [ ]:
# Make predictions with the test dataset
result = predictor.predict(X_test)
print(result)

In [ ]:
# Cleanup the endpoint
predictor.delete_endpoint()

# <font color='#8735fb'> Summary
    
In this Notebook, we took everything we have worked so hard on during this module and put it to use! We used the GPU-accelerated Forest Inference Library from RAPIDS to load our models, and then we ran a few basic benchmarks. Comparing the results from RAPIDS to those without RAPIDS, we saw that our XGBoost model ran an entire order of magnitude faster when accelerated by RAPIDS.

In this case, we have been preparing data, training models, and running inference all in the same development environment. But outside of the classroom, you would likely be doing this all in different environments, with different hardware resources, and possibly managed by different teams. If you review what you have learned so far, you will see that in each Notebook, we generated an artifact, be it raw data, processed data, or a trained model. In a real-world data science team, RAPIDS enables each team to perform these steps either in isolation or through a pipeline of services, and then hand off those artifacts. For example, as a data scientist, you have just trained a model and verified it loads perfectly into FIL. The next step would be to upload that model to a shared storage location, version control it, and hand it of to a DevOps or QA team to further integrate into an application and deploy into production!
    
This concludes our end-to-end workflow on this problem. In the next activity, we will explore the iterative workflow a bit more and see how we can combine some of these steps together using hyperparameter optimization (HPO) and Amazon SageMaker.

<center><a href="https://developer.nvidia.com/machine-learning"><img src="img/nvidia.png" alt="nvidia_logo" style="width: 1500px;"/></a></center>